In [1]:
import os
import pandas as pd
import xlrd
from datetime import datetime, date, timedelta

In [2]:
folder = 'c:\\Users\\erick\\Desktop\\Archivos'
lista_campos = ['Periodo', 'Region', 'Country', 'Item Type']
nombre_primera_columa = 'Region'
ruta_de_salida = 'c:\\Users\\erick\\Desktop\\'

In [3]:
def inicio_del_mes():
    #Devuelve el primer día del mes actual en formato de fecha
    return datetime.now().date().replace(day=1)

def anio():
    # Año actual en formato YY convertido a texto
    return str((inicio_del_mes() - timedelta(days=1)).year)[-2:]

def mes():
    # Mes anterior al actual en formato MM convertido a texto
    return ("00" + str((inicio_del_mes() - timedelta(days=1)).month))[-2:] 

def mes_ant():
    # Mes anterior a mes() en formato MM, el lag de 32 dias garantiza dos meses de retraso
    return ("00" + str((inicio_del_mes() - timedelta(days=32)).month))[-2:]

def valid_period(file_name):
    # Valida que el periodo del archivo se encuentre entre los dos meses anteriores 
    if file_name.split('.')[0][-5:][-2:] == anio() and file_name.split('.')[0][-5:][:2] in [mes(), mes_ant()]:
        return True
    else:
        return False
        
def valid_extension(file_name):
    # Valida si un archivo tiene la extensión correcta
    if file_name.endswith(".xls") or file_name.endswith(".xlsx"):
        return True
    elif file_name.endswith(".xlsb"):
        return True
    else:
        return False

def valid_file(file_name):
    # Comprueba que el archivo cumpla las tres condiciones para considerarse valido
    if valid_extension(file_name) and valid_period(file_name) and file_name.startswith("Nielsen"):
        return True
    else:
        return False

all_files = []
def list_of_files(file_path):
    # Crea una lista con los archivos del directorio que son validos en fecha y extensión
    for subfolder, folder, files in os.walk(file_path):
        [all_files.append(subfolder + os.sep + file) for file in files if valid_file(file)]

def export_to_csv(df, filename):
    # Guarda el DF en un archivo CSV en la ruta de la variable ruta_de_salida
    df.to_csv(ruta_de_salida + filename, index = False)

In [27]:
def delete_top_rows(dataframe):
        # Identifica en que fila se encuentra el primer encabezado (variable nombre_primera_columa) 
        top_row = dataframe.index[dataframe['Unnamed: 0'] == nombre_primera_columa].tolist()
        
        # Elimina las primeras filas que no tienen datos
        dataframe = dataframe.iloc[top_row[0]:]
        dataframe = dataframe.reset_index(drop=True)
        
        # Promover primera fila como encabezado de columna
        new_header = dataframe.iloc[0]
        dataframe = dataframe[1:]
        dataframe.columns = new_header 
        
        return dataframe

In [36]:
def select_columns(filename, old_df):
    # Toma del nombre del archivo el último periodo con datos
    row_value = filename.split('.')[0][-6:-3] + "'" + filename.split('.')[0][-2:]
    
    # Selecciona columnas especificadas en lista_campos + ultimo periodo
    old_df['Periodo'] = row_value
    old_df = old_df[lista_campos + [row_value]]
    old_df.rename(columns = {row_value:'Value'}, inplace = True) 
    
    return old_df

## Test

In [5]:
list_of_files(folder)

In [6]:
data = pd.DataFrame(columns=lista_campos + ['Value'])

In [8]:
all_files[0]

'c:\\Users\\erick\\Desktop\\Archivos\\Nielsen - Extraccion Papel Higienico Wm_Nacional y areas a P09 20.xlsb'

In [28]:
df = pd.read_excel(all_files[0], engine='pyxlsb')

In [29]:
df

,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13,Unnamed: 14,Unnamed: 15,Unnamed: 16,Unnamed: 17
0,Departamento: Papel higienico,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,fecha: septiembre de 2020,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Datos de Nielsen,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Region,Country,Item Type,Sales Channel,Order Priority,Order Date,Order ID,Ship Date,Units Sold,Unit Price,Unit Cost,Total Revenue,Total Cost,Total Profit,P06'20,P07'20,P08'20,P09'20
5,Middle East and North Africa,Azerbaijan,Snacks,Online,C,41920,535113847,41935,934,152.58,97.44,142510,91009,51500.8,1,2,3,4
6,Central America and the Caribbean,Panama,Cosmetics,Offline,L,42057,874708545,42062,4551,437.2,263.33,1.9897e+06,1.19841e+06,791282,2,3,4,5
7,Sub-Saharan Africa,Sao Tome and Principe,Fruits,Offline,M,42347,854349935,42387,9986,9.33,6.92,93169.4,69103.1,24066.3,1,2,3,4
8,Sub-Saharan Africa,Sao Tome and Principe,Personal Care,Online,M,41899,892836844,41924,9118,81.73,56.67,745214,516717,228497,2,3,4,5
9,Central America and the Caribbean,Belize,Household,Offline,H,40213,129280602,40242,5858,668.27,502.54,3.91473e+06,2.94388e+06,970846,1,2,3,4


In [30]:
df = delete_top_rows(df)

In [13]:
top_row = df.index[df['Unnamed: 0'] == nombre_primera_columa].tolist()

In [14]:
top_row

[4]

In [17]:
df = df.iloc[top_row[0]:]

In [20]:
df = df.reset_index(drop=True)

In [31]:
df

,Region,Country,Item Type,Sales Channel,Order Priority,Order Date,Order ID,Ship Date,Units Sold,Unit Price,Unit Cost,Total Revenue,Total Cost,Total Profit,P06'20,P07'20,P08'20,P09'20
1,Middle East and North Africa,Azerbaijan,Snacks,Online,C,41920,535113847,41935,934,152.58,97.44,142510,91009,51500.8,1,2,3,4
2,Central America and the Caribbean,Panama,Cosmetics,Offline,L,42057,874708545,42062,4551,437.2,263.33,1.9897e+06,1.19841e+06,791282,2,3,4,5
3,Sub-Saharan Africa,Sao Tome and Principe,Fruits,Offline,M,42347,854349935,42387,9986,9.33,6.92,93169.4,69103.1,24066.3,1,2,3,4
4,Sub-Saharan Africa,Sao Tome and Principe,Personal Care,Online,M,41899,892836844,41924,9118,81.73,56.67,745214,516717,228497,2,3,4,5
5,Central America and the Caribbean,Belize,Household,Offline,H,40213,129280602,40242,5858,668.27,502.54,3.91473e+06,2.94388e+06,970846,1,2,3,4
6,Europe,Denmark,Clothes,Online,C,41325,473105037,41333,1149,109.28,35.84,125563,41180.2,84382.6,2,3,4,5
7,Europe,Germany,Cosmetics,Offline,M,41364,754046475,41397,7964,437.2,263.33,3.48186e+06,2.09716e+06,1.3847e+06,1,2,3,4
8,Middle East and North Africa,Turkey,Fruits,Online,C,40994,772153747,41006,6307,9.33,6.92,58844.3,43644.4,15199.9,2,3,4,5
9,Europe,United Kingdom,Snacks,Online,H,41272,847788178,41289,8217,152.58,97.44,1.25375e+06,800664,453085,1,2,3,4
10,Asia,Kazakhstan,Cosmetics,Online,H,42258,471623599,42265,2758,437.2,263.33,1.2058e+06,726264,479533,2,3,4,5


In [37]:
df2 = select_columns(all_files[0], df)

c:\users\erick\appdata\local\programs\python\python38\lib\site-packages\pandas\core\frame.py:4296: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(


In [38]:
df2

,Periodo,Region,Country,Item Type,Value
1,P09'20,Middle East and North Africa,Azerbaijan,Snacks,4
2,P09'20,Central America and the Caribbean,Panama,Cosmetics,5
3,P09'20,Sub-Saharan Africa,Sao Tome and Principe,Fruits,4
4,P09'20,Sub-Saharan Africa,Sao Tome and Principe,Personal Care,5
5,P09'20,Central America and the Caribbean,Belize,Household,4
6,P09'20,Europe,Denmark,Clothes,5
7,P09'20,Europe,Germany,Cosmetics,4
8,P09'20,Middle East and North Africa,Turkey,Fruits,5
9,P09'20,Europe,United Kingdom,Snacks,4
10,P09'20,Asia,Kazakhstan,Cosmetics,5
